In [5]:
import pandas as pd
import sklearn
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report,  accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,average_precision_score, precision_recall_curve,auc
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.utils import pad_sequences
import tensorflow as tf

In [6]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def evaluation_metric(actual, pred):
    precision, recall, thresholds = precision_recall_curve(actual, pred)
    auc_precision_recall = auc(recall, precision)
    return (auc_precision_recall)

In [8]:
#Loading the training, validation and testing data
train=pd.read_csv("train.csv")
validation=pd.read_csv("validation.csv")
test=pd.read_csv("test.csv")

In [9]:
y_train,X_train=train["label"],train["message"]
y_val,X_val=validation["label"],vaidationl["message"]
y_test,X_test=validation["label"],validation["message"]

# Random Forest

In [10]:
#Using TfidfVectorizer with 600 features
tfidf=TfidfVectorizer(max_features=600)
encd_X_train=tfidf.fit_transform(X_train)

In [11]:
encd_X_val=tfidf.fit_transform(X_val)

In [12]:
encd_X_test=tfidf.fit_transform(X_test)

In [13]:
y_train=y_train.replace("spam",1)
y_train=y_train.replace("ham",0)

In [14]:
y_val=y_val.replace("spam",1)
y_val=y_val.replace("ham",0)

In [15]:
y_test=y_test.replace("spam",1)
y_test=y_test.replace("ham",0)

In [19]:
#Number of estimators
n=500

#Depth
depth=7

filename="rndmsrst.pkl"

with mlflow.start_run():
        rndmfrst = RandomForestClassifier(n_estimators=n,max_depth=depth, random_state=1)

        rndmfrst.fit(encd_X_train,y_train)

        pickle.dump(rndmfrst,open(filename,"wb"))

        y_pred=rndmfrst.predict(encd_X_test)


        res = evaluation_metric(y_test, y_pred)

        print("  AUPCR: %s" % res)
        mlflow.log_param("n_estimators", n)
        mlflow.log_param("max_depth", depth)
        mlflow.log_metric("aupcr", res)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        mlflow.sklearn.log_model(rndmfrst, "model", registered_model_name="RandomForestModel")

  AUPCR: 0.5639952153110048


Registered model 'RandomForestModel' already exists. Creating a new version of this model...
2023/05/01 10:26:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestModel, version 2
Created version '2' of model 'RandomForestModel'.


In [20]:
mlflow.tracking.MlflowClient().get_model_version("RandomForestModel","1")

<ModelVersion: aliases=[], creation_timestamp=1682916696790, current_stage='None', description=None, last_updated_timestamp=1682916696790, name='RandomForestModel', run_id='daa5b65f04bb447cac3f9efaaa5fd636', run_link=None, source='file:///E:/Assigments/MSc%20Semester%202/AML%20-%20Raghav/Programming%20assignment%203/mlruns/0/daa5b65f04bb447cac3f9efaaa5fd636/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

# Logistic Regression


In [18]:
from sentence_transformers import SentenceTransformer

#Pre-trained model to evaluate the embeddings of the text data
model = SentenceTransformer('all-mpnet-base-v2')

In [21]:
_,X_train=train["label"],train["message"]
_,X_val=validation["label"],validation["message"]
_,X_test=validation["label"],validation["message"]

In [22]:
embedded_X_train=model.encode(X_train)
embedded_X_test=model.encode(X_test)

In [23]:
max_iters=1000

In [24]:
filename="logreg.pkl"

In [25]:
with mlflow.start_run():
        logreg = LogisticRegression(max_iter=max_iters, random_state=1)

        logreg.fit(embedded_X_train,y_train)

        pickle.dump(logreg,open(filename,"wb"))

        y_pred=logreg.predict(embedded_X_test)


        res = evaluation_metric(y_test, y_pred)

        print("  AUPCR: %s" % res)
        mlflow.log_param("max_iter", n)
        
        mlflow.log_metric("aupcr", res)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        mlflow.sklearn.log_model(logreg, "model", registered_model_name="LogisticRegTransformers")

  AUPCR: 0.9407929887871673


Successfully registered model 'LogisticRegTransformers'.
2023/05/01 10:40:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: LogisticRegTransformers, version 1
Created version '1' of model 'LogisticRegTransformers'.


In [29]:
mlflow.tracking.MlflowClient().get_model_version("RandomForestModel","1")

<ModelVersion: aliases=[], creation_timestamp=1682916696790, current_stage='None', description=None, last_updated_timestamp=1682916696790, name='RandomForestModel', run_id='daa5b65f04bb447cac3f9efaaa5fd636', run_link=None, source='file:///E:/Assigments/MSc%20Semester%202/AML%20-%20Raghav/Programming%20assignment%203/mlruns/0/daa5b65f04bb447cac3f9efaaa5fd636/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>